In [1]:
import geopandas as gpd
import pandas as pd
from siuba import *

from segment_speed_utils import helpers, gtfs_schedule_wrangling
from shared_utils import rt_dates, gtfs_utils_v2
import folium

In [2]:
from update_vars import analysis_date, AM_PEAK, PM_PEAK, EXPORT_PATH

In [3]:
AM_PEAK

(datetime.time(6, 0), datetime.time(9, 0))

In [4]:
PM_PEAK

(datetime.time(15, 0), datetime.time(19, 0))

In [10]:
imported_st = helpers.import_scheduled_stop_times(
    analysis_date,
    get_pandas = True,
)

In [7]:
am_peak_hrs = list(range(AM_PEAK[0].hour, AM_PEAK[1].hour))
pm_peak_hrs = list(range(PM_PEAK[0].hour, PM_PEAK[1].hour))
both_peaks_hrs = am_peak_hrs + pm_peak_hrs
peaks_dict = {key: 'am_peak' for key in am_peak_hrs} | {key: 'pm_peak' for key in pm_peak_hrs}

#  happens in st agg...
imported_st = imported_st.assign(
        departure_hour = pd.to_datetime(
            imported_st.departure_sec, unit="s").dt.hour
    )

both_peak_filtered = imported_st[imported_st['arrival_hour'].isin(both_peaks_hrs)]
both_peak_filtered['peak'] = both_peak_filtered['arrival_hour'].map(peaks_dict)

In [8]:
peaks_dict

{6: 'am_peak',
 7: 'am_peak',
 8: 'am_peak',
 15: 'pm_peak',
 16: 'pm_peak',
 17: 'pm_peak',
 18: 'pm_peak'}

## sjoin stops to segs -- subset peak?

In [9]:
import sjoin_stops_to_segments

In [12]:
imported_st.dtypes

feed_key           object
feed_timezone      object
base64_url         object
trip_id            object
stop_id            object
stop_sequence       int64
timepoint         float64
arrival_sec       float64
departure_sec     float64
arrival_hour      float64
departure_hour    float64
dtype: object

In [13]:
imported_st >> head(3)

,feed_key,feed_timezone,base64_url,trip_id,stop_id,stop_sequence,timepoint,arrival_sec,departure_sec,arrival_hour,departure_hour
0,55d8763ca8845a4efa50e40a9b972e1c,America/Los_Angeles,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,dgxbvk5qu,11,1,1.0,43200.0,43200.0,12.0,12.0
1,55d8763ca8845a4efa50e40a9b972e1c,America/Los_Angeles,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,992,GA,1,1.0,85200.0,85200.0,23.0,23.0
2,55d8763ca8845a4efa50e40a9b972e1c,America/Los_Angeles,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,ogkycex13,11,1,1.0,15000.0,15000.0,4.0,4.0


In [16]:
both_peak_filtered >> head(2)

,feed_key,feed_timezone,base64_url,trip_id,stop_id,stop_sequence,timepoint,arrival_sec,departure_sec,arrival_hour,departure_hour,peak
3,55d8763ca8845a4efa50e40a9b972e1c,America/Los_Angeles,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,hcm9spuol,11,1,1.0,59400.0,59400.0,16.0,16.0,pm_peak
5,55d8763ca8845a4efa50e40a9b972e1c,America/Los_Angeles,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,1335,LTP,1,1.0,64080.0,64080.0,17.0,17.0,pm_peak


In [17]:
# (1) Aggregate stop times - by stop_id, find max trips in AM/PM peak
# takes 1 min
max_arrivals_by_stop = both_peak_filtered.pipe(sjoin_stops_to_segments.stop_times_aggregation_max_by_stop, analysis_date)

In [18]:
#  peak filtered but old methodology...
max_arrivals_by_stop >> head(2)

,schedule_gtfs_dataset_key,stop_id,am_max_trips,pm_max_trips,n_trips
0,0139b1253130b33adcd4b3a4490530d2,00eb15cb-1430-4964-b8ae-ca6183e1d0ef,1,1,2.0
1,0139b1253130b33adcd4b3a4490530d2,02a30e39-496f-45d4-ba1c-ac8f3c66b621,3,4,7.0


In [19]:
def dev_stop_times_aggregation_max_by_stop(
    stop_times: pd.DataFrame, 
    analysis_date: str,
    single_route_dir: bool = False,
) -> pd.DataFrame:
    """
    Take the stop_times table 
    and group by stop_id-departure hour
    and count how many trips occur.
    """
    stop_cols = ["schedule_gtfs_dataset_key", "stop_id"]
    trips_per_hour_cols = ["peak"]
    
    if single_route_dir:
        trips_per_hour_cols += ["route_id", "direction_id"]

    trips = helpers.import_scheduled_trips(
        analysis_date,
        columns = ["feed_key", "gtfs_dataset_key", "trip_id",
                   "route_id", "direction_id"],
        get_pandas = True
    )

    stop_times = both_peak_filtered.assign(
        departure_hour = pd.to_datetime(
            both_peak_filtered.departure_sec, unit="s").dt.hour
    ).merge(
        trips,
        on = ["feed_key", "trip_id"]
    )
    # display(stop_times.head(2))
            
    # Aggregate how many trips are made at that stop by departure hour
    trips_per_hour = gtfs_schedule_wrangling.stop_arrivals_per_stop(
        stop_times,
        group_cols = stop_cols + trips_per_hour_cols, #  TODO update script
        count_col = "trip_id"
    ).rename(columns = {"n_arrivals": "n_trips"})
    
    # return trips_per_hour
    
    # Subset to departure hour before or after 12pm
    am_trips = sjoin_stops_to_segments.max_trips_by_group(
        trips_per_hour[trips_per_hour.peak == 'am_peak'], 
        group_cols = stop_cols,
        max_col = "n_trips"
    ).rename(columns = {"n_trips": "am_max_trips"})
    
    pm_trips = sjoin_stops_to_segments.max_trips_by_group(
        trips_per_hour[trips_per_hour.peak == 'pm_peak'], 
        group_cols = stop_cols,
        max_col = "n_trips"
    ).rename(columns = {"n_trips": "pm_max_trips"})
    
    max_trips_by_stop = pd.merge(
        am_trips, 
        pm_trips,
        on = stop_cols,
        how = "left"
    )
    #  divide by length of peak to get trips/hr, keep n_trips a raw sum
    max_trips_by_stop = max_trips_by_stop.assign(
        am_max_trips_hr = (max_trips_by_stop.am_max_trips.fillna(0) / len(am_peak_hrs)).astype(int),
        pm_max_trips_hr = (max_trips_by_stop.pm_max_trips.fillna(0) / len(pm_peak_hrs)).astype(int),
        n_trips = (max_trips_by_stop.am_max_trips.fillna(0) + 
                   max_trips_by_stop.pm_max_trips.fillna(0))
    )
        
    return max_trips_by_stop

In [20]:
test = dev_stop_times_aggregation_max_by_stop(both_peak_filtered, analysis_date, single_route_dir=False)

In [21]:
old_hq = max_arrivals_by_stop >> filter(_.am_max_trips > 4, _.pm_max_trips > 4) # old HQ corridor + major stop precursor
old_hq

,schedule_gtfs_dataset_key,stop_id,am_max_trips,pm_max_trips,n_trips
96,0139b1253130b33adcd4b3a4490530d2,52c2636c-34a3-434c-99ae-cdf3dc36d15c,22,28,50.0
179,0139b1253130b33adcd4b3a4490530d2,98d2a60c-86b1-45d6-b5d6-39b273c9eb46,8,8,16.0
180,0139b1253130b33adcd4b3a4490530d2,999ff07b-4a27-4c80-9a1c-e868038ce097,13,12,25.0
250,0139b1253130b33adcd4b3a4490530d2,e37ef534-cce7-4470-84b4-25ba98c51114,5,5,10.0
296,015d67d5b75b5cf2b710bbadadfb75f5,40103,5,5,10.0
...,...,...,...,...,...
77910,ff1bc5dde661d62c877165421e9ca257,LO_1,5,5,10.0
77916,ff1bc5dde661d62c877165421e9ca257,LO_15,5,5,10.0
77918,ff1bc5dde661d62c877165421e9ca257,LO_17,5,5,10.0
77919,ff1bc5dde661d62c877165421e9ca257,LO_18,5,5,10.0


In [22]:
new_hq_multi = test >> filter(_.am_max_trips_hr > 4, _.pm_max_trips_hr > 4) #  new HQ corridor (still multi-route)
new_hq_multi

,schedule_gtfs_dataset_key,stop_id,am_max_trips,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips
96,0139b1253130b33adcd4b3a4490530d2,52c2636c-34a3-434c-99ae-cdf3dc36d15c,39,58.0,13,14,97.0
179,0139b1253130b33adcd4b3a4490530d2,98d2a60c-86b1-45d6-b5d6-39b273c9eb46,16,24.0,5,6,40.0
180,0139b1253130b33adcd4b3a4490530d2,999ff07b-4a27-4c80-9a1c-e868038ce097,25,44.0,8,11,69.0
297,015d67d5b75b5cf2b710bbadadfb75f5,40113,30,48.0,10,12,78.0
332,015d67d5b75b5cf2b710bbadadfb75f5,40158,21,34.0,7,8,55.0
...,...,...,...,...,...,...,...
77668,fb467982dcc77a7f9199bebe709bb700,66001,20,27.0,6,6,47.0
77695,fb746afc72ff40405cfefa6d23ab58a0,53000,22,27.0,7,6,49.0
77703,fb746afc72ff40405cfefa6d23ab58a0,53129,28,42.0,9,10,70.0
77705,fb746afc72ff40405cfefa6d23ab58a0,53163,20,27.0,6,6,47.0


In [23]:
new_ms_multi = test >> filter(_.am_max_trips_hr > 3, _.pm_max_trips_hr > 3) #  new major stop precursor (still multi-route)
new_ms_multi

,schedule_gtfs_dataset_key,stop_id,am_max_trips,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips
96,0139b1253130b33adcd4b3a4490530d2,52c2636c-34a3-434c-99ae-cdf3dc36d15c,39,58.0,13,14,97.0
179,0139b1253130b33adcd4b3a4490530d2,98d2a60c-86b1-45d6-b5d6-39b273c9eb46,16,24.0,5,6,40.0
180,0139b1253130b33adcd4b3a4490530d2,999ff07b-4a27-4c80-9a1c-e868038ce097,25,44.0,8,11,69.0
296,015d67d5b75b5cf2b710bbadadfb75f5,40103,12,17.0,4,4,29.0
297,015d67d5b75b5cf2b710bbadadfb75f5,40113,30,48.0,10,12,78.0
...,...,...,...,...,...,...,...
77703,fb746afc72ff40405cfefa6d23ab58a0,53129,28,42.0,9,10,70.0
77705,fb746afc72ff40405cfefa6d23ab58a0,53163,20,27.0,6,6,47.0
77749,fc6cd27871cce0092a08ccf68fb240a2,132744,19,23.0,6,5,42.0
77772,fe4aab1717eca5a2935c32c85a35a5bf,115,13,22.0,4,5,35.0


In [24]:
single_route_test = dev_stop_times_aggregation_max_by_stop(both_peak_filtered, analysis_date, single_route_dir=True)

In [25]:
new_hq_single = single_route_test >> filter(_.am_max_trips_hr > 4, _.pm_max_trips_hr > 4) #  new HQ corridor (single-route)
new_hq_single

,schedule_gtfs_dataset_key,stop_id,am_max_trips,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips
560,0666caf3ec1ecc96b74f4477ee4bc939,10089,18,21.0,6,5,39.0
563,0666caf3ec1ecc96b74f4477ee4bc939,10091,18,21.0,6,5,39.0
565,0666caf3ec1ecc96b74f4477ee4bc939,10093,18,21.0,6,5,39.0
568,0666caf3ec1ecc96b74f4477ee4bc939,10096,15,20.0,5,5,35.0
569,0666caf3ec1ecc96b74f4477ee4bc939,10097,16,20.0,5,5,36.0
...,...,...,...,...,...,...,...
74870,fb467982dcc77a7f9199bebe709bb700,65047,16,20.0,5,5,36.0
75116,fb467982dcc77a7f9199bebe709bb700,65850,19,24.0,6,6,43.0
75128,fb467982dcc77a7f9199bebe709bb700,65874,18,24.0,6,6,42.0
75129,fb467982dcc77a7f9199bebe709bb700,65875,17,24.0,5,6,41.0


In [26]:
new_ms_single = single_route_test >> filter(_.am_max_trips_hr > 3, _.pm_max_trips_hr > 3) #  new major stop precursor (single-route)
new_ms_single

,schedule_gtfs_dataset_key,stop_id,am_max_trips,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips
560,0666caf3ec1ecc96b74f4477ee4bc939,10089,18,21.0,6,5,39.0
563,0666caf3ec1ecc96b74f4477ee4bc939,10091,18,21.0,6,5,39.0
565,0666caf3ec1ecc96b74f4477ee4bc939,10093,18,21.0,6,5,39.0
568,0666caf3ec1ecc96b74f4477ee4bc939,10096,15,20.0,5,5,35.0
569,0666caf3ec1ecc96b74f4477ee4bc939,10097,16,20.0,5,5,36.0
...,...,...,...,...,...,...,...
75139,fb467982dcc77a7f9199bebe709bb700,65890,13,16.0,4,4,29.0
75151,fb467982dcc77a7f9199bebe709bb700,65906,15,20.0,5,5,35.0
75152,fb467982dcc77a7f9199bebe709bb700,65907,12,16.0,4,4,28.0
75202,fb467982dcc77a7f9199bebe709bb700,65970,13,16.0,4,4,29.0


## preliminary mapping

* add bus/rail/ferry MTS in orange to help decide

In [27]:
EXPORT_PATH

'gs://calitp-analytics-data/data-analyses/high_quality_transit_areas/export/2024-09-18/'

In [28]:
hq_points = gpd.read_parquet(f'{EXPORT_PATH}ca_hq_transit_stops.parquet')

In [29]:
non_bus = hq_points >> filter(-_.hqta_type.isin(['hq_corridor_bus', 'major_stop_bus']))

In [30]:
both_peak_filtered

,feed_key,feed_timezone,base64_url,trip_id,stop_id,stop_sequence,timepoint,arrival_sec,departure_sec,arrival_hour,departure_hour,peak
3,55d8763ca8845a4efa50e40a9b972e1c,America/Los_Angeles,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,hcm9spuol,11,1,1.0,59400.0,59400.0,16.0,16.0,pm_peak
5,55d8763ca8845a4efa50e40a9b972e1c,America/Los_Angeles,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,1335,LTP,1,1.0,64080.0,64080.0,17.0,17.0,pm_peak
6,55d8763ca8845a4efa50e40a9b972e1c,America/Los_Angeles,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,783,GA,1,1.0,22500.0,22500.0,6.0,6.0,am_peak
15,55d8763ca8845a4efa50e40a9b972e1c,America/Los_Angeles,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,14c6ustn3,11,1,1.0,63000.0,63000.0,17.0,17.0,pm_peak
18,55d8763ca8845a4efa50e40a9b972e1c,America/Los_Angeles,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,xeg022osx,11,1,1.0,55800.0,55800.0,15.0,15.0,pm_peak
...,...,...,...,...,...,...,...,...,...,...,...,...
3810549,4beca22fda0d902da1416210ec025560,US/Pacific,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,309020,50500,47,1.0,27180.0,27180.0,7.0,7.0,am_peak
3810550,4beca22fda0d902da1416210ec025560,US/Pacific,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,13315020,51067,47,1.0,58200.0,58200.0,16.0,16.0,pm_peak
3810551,4beca22fda0d902da1416210ec025560,US/Pacific,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,9294020,55522,47,1.0,57120.0,57120.0,15.0,15.0,pm_peak
3810552,4beca22fda0d902da1416210ec025560,US/Pacific,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,3207020,55522,47,1.0,58260.0,58260.0,16.0,16.0,pm_peak


In [31]:
gtfs_utils_v2.get_stops?

Signature:
gtfs_utils_v2.get_stops(
    selected_date: Union[str, datetime.date],
    operator_feeds: list[str] = [],
    stop_cols: list[str] = [],
    get_df: bool = True,
    crs: str = 'EPSG:4326',
    custom_filtering: dict = None,
) -> Union[geopandas.geodataframe.GeoDataFrame, siuba.sql.backend.LazyTbl]
Docstring:
Query fct_daily_scheduled_stops.

Must supply a list of feed_keys or organization names returned from
schedule_daily_feed_to_gtfs_dataset_name() or subset of those results.
File:      ~/data-analyses/_shared_utils/shared_utils/gtfs_utils_v2.py
Type:      function

In [32]:
stops = gtfs_utils_v2.get_stops(selected_date=analysis_date, operator_feeds=both_peak_filtered.feed_key.unique(),
                       stop_cols=['feed_key', 'stop_id'])

/opt/conda/lib/python3.9/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_geom'
  sqlalchemy.util.warn(


In [33]:
feeds = gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(selected_date=analysis_date) >> select(_.feed_key, _.schedule_gtfs_dataset_key == _.gtfs_dataset_key)

In [34]:
stops = stops >> inner_join(_, feeds, on='feed_key')

In [35]:
def add_to_map(test_df, m=None, **kwargs):
    gdf = stops >> inner_join(_, test_df, on=['schedule_gtfs_dataset_key', 'stop_id'])
    m = gdf.explore(m = m, **kwargs)
    return m

In [36]:
m = add_to_map(old_hq)
m = non_bus.explore(m = m, color='orange')

In [37]:
m = add_to_map(new_hq_multi, m=m, color='green')

In [38]:
m = add_to_map(new_hq_single, m=m, color='red')

In [39]:
folium.LayerControl().add_to(m)

In [46]:
# m

## major stops

In [47]:
m = add_to_map(old_hq)
m = non_bus.explore(m = m, color='orange')

In [48]:
m = add_to_map(new_ms_multi, m=m, color='green')

In [49]:
m = add_to_map(new_ms_single, m=m, color='red')

In [50]:
folium.LayerControl().add_to(m)

In [52]:
# m